# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile, visualise
import os, glob
from tqdm.auto import tqdm
from skimage import io
import numpy as np
from macrohet.notify import send_sys_message as notify
import zarr

In [2]:
os.getpid()

391938

In [3]:
def update_slider(event):
    # only trigger if update comes from first axis (optional
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]
    viewer.text_overlay.text = f"{time*time_scale_factor:1.1f} hours"
text_size = 24
napari_scale = [1.4949402023919043e-07, 1.4949402023919043e-07]
label_text_size = 15
### glimpse size
size = 500
### resized images are to this scale
scale = 6048/1200
scale_factor = 1/5.04
from napari_animation import Animation


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [4]:
%%time
expt_ID = 'ND0004'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'
base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
# metadata_fn = '/mnt/SYNO/macrohet_syno/ND0001/acquisition/ND0001__2023-10-27T14_12_54-Measurement 1/Images/Index.idx.xml'#glob.glob(os.path.join(base_dir, 'Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
# temporary hack to fix URL from incorrectly exported metadata
# metadata['URL'] = metadata.apply(dataio.generate_url, axis=1)
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 32.5 s, sys: 3.06 s, total: 35.6 s
Wall time: 47.7 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0301K1F1P1R1,Ok,r03c01f01p01-ch1sk1fk1fl1.tiff,3,1,1,1,0,1,1,...,0,0.135243401,0,2024-04-17T15:13:28.903+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
1,0301K1F1P1R2,Ok,r03c01f01p01-ch2sk1fk1fl1.tiff,3,1,1,1,0,2,1,...,0,0.135243401,0,2024-04-17T15:13:29.123+01:00,488,522,40,1.1,0.1,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
2,0301K1F1P2R1,Ok,r03c01f01p02-ch1sk1fk1fl1.tiff,3,1,1,2,0,1,1,...,2E-06,0.135245398,0,2024-04-17T15:13:29.467+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
3,0301K1F1P2R2,Ok,r03c01f01p02-ch2sk1fk1fl1.tiff,3,1,1,2,0,2,1,...,2E-06,0.135245398,0,2024-04-17T15:13:29.7+01:00,488,522,40,1.1,0.1,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
4,0301K1F1P3R1,Ok,r03c01f01p03-ch1sk1fk1fl1.tiff,3,1,1,3,0,1,1,...,4E-06,0.135247394,0,2024-04-17T15:13:30.043+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349267,0612K154F9P1R2,Ok,r06c12f09p01-ch2sk154fk1fl1.tiff,6,12,9,1,153,2,1,...,0,0.1350099,275404.583,2024-04-20T19:57:46.98+01:00,488,522,40,1.1,0.1,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
349268,0612K154F9P2R1,Ok,r06c12f09p02-ch1sk154fk1fl1.tiff,6,12,9,2,153,1,1,...,2E-06,0.135011896,275404.583,2024-04-20T19:57:47.323+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
349269,0612K154F9P2R2,Ok,r06c12f09p02-ch2sk154fk1fl1.tiff,6,12,9,2,153,2,1,...,2E-06,0.135011896,275404.583,2024-04-20T19:57:47.557+01:00,488,522,40,1.1,0.1,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."
349270,0612K154F9P3R1,Ok,r06c12f09p03-ch1sk154fk1fl1.tiff,6,12,9,3,153,1,1,...,4E-06,0.135013893,275404.583,2024-04-20T19:57:47.9+01:00,640,706,40,1.1,0.2,"[[0.999619,0,0,-5.1],[0,-0.999619,0,-2.5],[0,0..."


In [5]:
time_scale_factor = round(float(metadata[metadata['TimepointID']=='1']['MeasurementTimeOffset'].iloc[0])/60/60, 1)

### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [6]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*AssayLayout.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC  Replicate #
Row Column                                                           
3   1         UNI     CTRL             0             EC0            1
    2         UNI     CTRL             0             EC0            2
    3          WT     CTRL             0             EC0            1
    4          WT     CTRL             0             EC0            2
    5          WT      PZA            60            EC50            1
    6          WT      PZA            60            EC50            2
    7          WT      RIF           0.1            EC50            1
    8          WT      RIF           0.1            EC50            2
    9          WT      INH          0.04            EC50            1
    10         WT      INH          0.04            EC50            2
    11         WT      BDQ          0.02            EC50            1
    12         WT      BDQ          0.02            EC50            2
4   3          WT     CTRL             0             EC0            3
    4          WT     CTRL             0             EC0            4
    5          WT      PZA           400            EC99            1
    6          WT      PZA           400            EC99            2
    7          WT      RIF             2            EC99            1
    8          WT      RIF             2            EC99            2
    9          WT      INH             2            EC99            1
    10         WT      INH             2            EC99            2
    11         WT      BDQ           2.5            EC99            1
    12         WT      BDQ           2.5            EC99            2
5   3         RD1     CTRL             0             EC0            1
    4         RD1     CTRL             0             EC0            2
    5         RD1      PZA            60            EC50            1
    6         RD1      PZA            60            EC50            2
    7         RD1      RIF           0.1            EC50            1
    8         RD1      RIF           0.1            EC50            2
    9         RD1      INH          0.04            EC50            1
    10        RD1      INH          0.04            EC50            2
    11        RD1      BDQ          0.02            EC50            1
    12        RD1      BDQ          0.02            EC50            2
6   3         RD1     CTRL             0             EC0            3
    4         RD1     CTRL             0             EC0            4
    5         RD1      PZA           400            EC99            1
    6         RD1      PZA           400            EC99            2
    7         RD1      RIF             2            EC99            1
    8         RD1      RIF             2            EC99            2
    9         RD1      INH             2            EC99            1
    10        RD1      INH             2            EC99            2
    11        RD1      BDQ           2.5            EC99            1
    12        RD1      BDQ           2.5            EC99            2

# Set size of napari viewer first

In [10]:
viewer = napari.Viewer()

In [11]:
viewer.add_image(images_max_proj, channel_axis = 1, 
                         scale = napari_scale, contrast_limits=[[280, 1000],[0,3000]])

[<Image layer 'Image' at 0x7ff62bd764f0>,
 <Image layer 'Image [1]' at 0x7ff622ee7d90>]

In [12]:
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'white'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

<function __main__.update_slider(event)>

### Load images using zarr and save out mp4

In [36]:
for acq_ID, data in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    try:
        data_string = f'{data.values[0]}_{data.values[1]}_{data.values[3]}'
        output_fn = f'/mnt/SYNO/videos/macrohet_videos/{expt_ID}/{expt_ID}_{acq_ID}_{data_string}.mp4'
        if os.path.exists(output_fn):
            print('skipping', output_fn)
            continue
        # image_dir = os.path.join(base_dir, 'acquisition/Images')
        row, column = acq_ID
#         images_max_proj = tile.compile_mosaic(image_dir, 
#                              metadata, 
#                              row, column, 
#                              set_plane='max_proj',
#                              # set_channel=1,
#                              # set_time = 1,
# #                             input_transforms = [input_transforms]
#                             ).compute().compute()
        
        zarr_store = zarr.open(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/zarr/{acq_ID}.zarr')
        zarr_images = zarr_store.images #[]
        
        images_max_proj = np.max(zarr_images, axis = 2)
        
        viewer = napari.Viewer(title = f'{expt_ID, acq_ID} mp4 gen')
        
        viewer.add_image(images_max_proj, channel_axis = 1, 
                         scale = napari_scale, contrast_limits=[[280, 1000],[0,3000]])
        # viewer.theme = 'light'
        viewer.scale_bar.visible = True
        viewer.scale_bar.unit = 'm'
        viewer.scale_bar.font_size = text_size
        viewer.text_overlay.visible = True
        viewer.text_overlay.color = 'white'
        viewer.text_overlay.position = 'bottom_left'
        viewer.text_overlay.font_size = text_size
        viewer.dims.events.current_step.connect(update_slider)
        
        # Create an animation object
        animation = Animation(viewer)
        viewer.dims.current_step = (0, 3023, 3023)
        # Capture the initial frame
        animation.capture_keyframe()
        viewer.dims.current_step = (150, 3023, 3023)
        # Capture the last frame (no need to change camera angles)
        animation.capture_keyframe(steps =600)
        
        # Animate to create a static video (from the first frame to the last frame)
        animation.animate(output_fn, canvas_only=True)
        
        viewer.close()
        notify(notification_message=f'{acq_ID} mp4 saved out')
    except:
        print(acq_ID, 'error')

  0%|          | 0/42 [00:00<?, ?it/s]

skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 1)_UNI_CTRL_EC0.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 2)_UNI_CTRL_EC0.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 3)_WT_CTRL_EC0.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 4)_WT_CTRL_EC0.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 5)_WT_PZA_EC50.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 6)_WT_PZA_EC50.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 7)_WT_RIF_EC50.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 8)_WT_RIF_EC50.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 9)_WT_INH_EC50.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 10)_WT_INH_EC50.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 11)_WT_BDQ_EC50.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 12)_WT_BDQ_EC50.mp4
skipping /mnt/SYNO/videos/macrohet_videos/ND000


  0%|                                                                | 0/601 [00:00<?, ?it/s]WARNING:imageio_ffmpeg:IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1033, 1055) to (1040, 1056) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

  0%|                                                        | 1/601 [00:00<06:35,  1.52it/s][swscaler @ 0x5614f5acfcc0] Warning: data is not aligned! This can lead to a speed loss

100%|██████████████████████████████████████████████████████| 601/601 [00:35<00:00, 17.02it/s]


Rendering frames...



  0%|                                                                | 0/601 [00:00<?, ?it/s]WARNING:imageio_ffmpeg:IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1033, 1055) to (1040, 1056) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

  0%|                                                        | 1/601 [00:00<06:20,  1.58it/s][swscaler @ 0x555e22890cc0] Warning: data is not aligned! This can lead to a speed loss

100%|██████████████████████████████████████████████████████| 601/601 [00:35<00:00, 16.82it/s]


...


  0%|                                                                | 0/601 [00:00<?, ?it/s]WARNING:imageio_ffmpeg:IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1033, 1055) to (1040, 1056) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

  0%|                                                        | 1/601 [00:00<06:45,  1.48it/s][swscaler @ 0x5589428c2cc0] Warning: data is not aligned! This can lead to a speed loss

100%|██████████████████████████████████████████████████████| 601/601 [00:35<00:00, 16.92it/s]


Rendering frames...



  0%|                                                                | 0/601 [00:00<?, ?it/s]WARNING:imageio_ffmpeg:IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1033, 1055) to (1040, 1056) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

  0%|                                                        | 1/601 [00:00<06:13,  1.60it/s][swscaler @ 0x55bb84f19cc0] Warning: data is not aligned! This can lead to a speed loss

100%|██████████████████████████████████████████████████████| 601/601 [00:31<00:00, 18.83it/s]


Rendering frames...



  0%|                                                                | 0/601 [00:00<?, ?it/s]WARNING:imageio_ffmpeg:IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1033, 1055) to (1040, 1056) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

  0%|                                                        | 1/601 [00:00<06:14,  1.60it/s][swscaler @ 0x55808176dcc0] Warning: data is not aligned! This can lead to a speed loss

100%|██████████████████████████████████████████████████████| 601/601 [00:32<00:00, 18.56it/s]


In [37]:
import os
import ffmpeg

def set_metadata_to_filename(input_file):

    filename = os.path.splitext(os.path.basename(input_file))[0]
    temp_output_file = input_file + '.temp.mp4'
    info = filename.split('_')
    title = f'Experiment:{info[0]}{info[1]}, strain:{info[2]}, compound:{info[3]}, concentration:{info[4]}'
    
    try:
        ffmpeg.input(input_file).output(temp_output_file, metadata=f'title={title}', c='copy').run()
        print(f"Updated metadata for: {input_file}")
        # Replace the original file with the new file
        os.replace(temp_output_file, input_file)
    except ffmpeg.Error as e:
        print(f"Failed to update metadata for: {input_file}, Error: {str(e)}")
        # Remove temp file if error occurs
        if os.path.exists(temp_output_file):
            os.remove(temp_output_file)
    
def update_metadata_in_directory(base_dir):
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(('EC0.mp4', 'EC50.mp4', 'EC99.mp4')):
                file_path = os.path.join(root, file)
                try:
                    set_metadata_to_filename(file_path)
                    print(f"Updated metadata for: {file_path}")
                except Exception as e:
                    print(f"Failed to update metadata for: {file_path}, Error: {str(e)}")

# Base directory containing the MP4 files
base_directory = '/mnt/SYNO/videos/macrohet_videos/ND0004'
update_metadata_in_directory(base_directory)



ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 1)_UNI_CTRL_EC0.mp4
Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 1)_UNI_CTRL_EC0.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 2)_UNI_CTRL_EC0.mp4
Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(3, 2)_UNI_CTRL_EC0.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

...

Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(6, 10)_RD1_INH_EC99.mp4
Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(6, 10)_RD1_INH_EC99.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(6, 11)_RD1_BDQ_EC99.mp4
Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(6, 11)_RD1_BDQ_EC99.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1705436738391/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(6, 12)_RD1_BDQ_EC99.mp4
Updated metadata for: /mnt/SYNO/videos/macrohet_videos/ND0004/ND0004_(6, 12)_RD1_BDQ_EC99.mp4
